In [1]:
from pyathena import connect
import csv

import pandas as pd
from datetime import datetime
import os
from dotenv import load_dotenv

In [2]:
env_src = "..//open-data-webapp//"

load_dotenv(env_src+"loc.env")

True

In [3]:
id = os.getenv("AWS_ACCESS_KEY")
pwd = os.getenv("AWS_SECRET_KEY")
bucket = os.getenv("S3_STAGING_DIR")
region = os.getenv("AWS_REGION")
schema = os.getenv("SCHEMA_NAME")
db = os.getenv("DATABASE_NAME")
table_name = os.getenv("TABLE_NAME")

In [4]:
engine = connect(
    aws_access_key_id=os.getenv("AWS_ACCESS_KEY"),
    aws_secret_access_key=os.getenv("AWS_SECRET_KEY"),
    region_name=os.getenv("AWS_REGION"),
    schema_name=os.getenv("SCHEMA_NAME"),
    s3_staging_dir=os.getenv("S3_STAGING_DIR"))

In [5]:
cursor = engine.cursor()

In [6]:
date_val = ""

In [7]:
val = cursor.execute("""SELECT * FROM default.nhm_order_fulfillment_subset_v1 
                     WHERE date(date_parse("O_Created Date & Time", '%%Y-%%m-%%dT%%H:%%i:%%s')) = DATE %(day_val)s""", 
                     {"day_val": "2024-01-01"})

In [8]:
data = val.fetchall()

Testing Python File Write

In [9]:
with open("out.csv",'w', newline='',encoding='utf8') as f:
  writer = csv.writer(f)
  writer.writerows(data)

Testing Writing with Pandas

In [10]:
pd.DataFrame(data).to_csv("jan.csv", index=False)

Testing writing data with Parquet

In [14]:
pd.DataFrame(data).to_parquet("jan_z.parquet",compression='zstd')

In [12]:
engine.close()

- It is faster to convert the dataset from Query object to csv using Python file method. 
- It is easier and memory efficient to convert csv into parquet with zstd compression using pandas. Can do this manually however any change in column name/ columns will require those columns to be identified manually. 
- Best method is to read the data into query format, write to csv using python file handling, and convert it to parquet using pandas. Then load it into the target database. 
- Each step can branch out into multiple threads for faster parallel processing. 